# Particle Filters 
are a sequence of algorithms for estimating the state of a system.

<img src="L3/1.png" alt="Drawing" style="width: 400px;"/>
* sonar senser is used in the figure.
* here is global localization, but particle filter can be used for other purposes?!

<img src="L3/2.png" alt="Drawing" style="width: 500px;"/>

* Histogram filter is exponential. Grid defined over k-dimensions.
* Kalman filter is quadratic. It is fully represented by a vector - the mean - and the covariance matrix, which is quadratic.

* Histogram filters are approximate because the world is not discrete; localization for example, is an approximate filter. 
* Kalman filters are also approximate, they are only exact for linear systems, however, the world is non-linear.

# Robot code

In [2]:
# Bug in the move method: When the noise is non-zero,
# everytime the robot makes a turn only. 
# The (x, y) is updated because of the forward_noise. verse vice!!!

from math import *
import random

landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError, 'X coordinate out of bound'
        if new_y < 0 or new_y >= world_size:
            raise ValueError, 'Y coordinate out of bound'
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError, 'Robot cant move backwards'         
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        ### Bug!
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        # calculates how likely a measurement should be        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))



In [3]:
myrobot = robot()
# enter code here
myrobot.set(30.0, 50.0, pi/2)
myrobot.set_noise(5.0, 0.1, 5.0)

myrobot = myrobot.move(-pi/2, 15.0)
print myrobot.sense()

myrobot = myrobot.move(-pi/2, 10.0)
print myrobot.sense()

[33.538533201583036, 48.80343471238755, 24.88374825472535, 50.982325827427815]
[25.90883576216007, 60.2308051623908, 60.395726667226555, 38.39442549940027]


# Robot world

<img src="L3/3.png" alt="Drawing" style="width: 400px;"/>
* note: assume the robot world is cyclic.

<img src="L3/4.png" alt="Drawing" style="width: 300px;"/>

<img src="L3/5.png" alt="Drawing" style="width: 300px;"/>

<img src="L3/6.png" alt="Drawing" style="width: 300px;"/>

<img src="L3/7.png" alt="Drawing" style="width: 300px;"/>
* note: the weight is the mismatch between the actual measurement and the predicted measurement.

<img src="L3/8.png" alt="Drawing" style="width: 300px;"/>

* Draws N new particles with replacement by picking each particle with a probability proportional to the value of it’s $\alpha_N = \frac{w_N}{W}$, where $W = \sum_i w_i$ and $w_i$ is the weight of each particle calculated by the **measurement_prob** method.

<img src="L3/9.png" alt="Drawing" style="width: 400px;"/>
* $\alpha_1+\alpha_2+\alpha_4+\alpha_5 = 0.6$, Then probablity of missing p3 is $0.6^5 = 0.0777$

In [18]:
N = 1000 # numbers of particles
T = 10 # resampling times

# create my robot
myrobot = robot()

# create particles
p = []
for i in range(N):
    x = robot() # random particles
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

# start the particle filter
for t in range(T):
    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()
    
    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2
    
    # calculate the weight
    w = []
    for i in range(N):
        weight = p[i].measurement_prob(Z)
        w.append(weight)
        
    # resampling
    p3=[] 
    index = int(random.random()*N) 
    beta = 0.0
    mw = max(w)
    for i in range(N):
        beta += random.random()*2.0*mw 
        while beta > w[index]:
            beta -= w[index]
            index = (index + 1)% N 
        p3.append(p[index])
    p=p3

    print eval(myrobot, p)

print p


4.46664443109
4.02442201618
1.85229837185
1.35435675085
1.21319337241
1.44503787449
1.72639564709
1.89700320175
1.97768767694
2.0531571378
[[x=31.776 y=32.626 orient=2.9165], [x=31.574 y=33.235 orient=2.8557], [x=29.563 y=28.419 orient=3.1351], [x=29.888 y=31.041 orient=2.8799], [x=31.346 y=32.752 orient=2.8728], [x=30.565 y=30.970 orient=2.9865], [x=28.836 y=28.205 orient=2.9516], [x=29.326 y=27.520 orient=3.1425], [x=31.296 y=32.245 orient=2.7945], [x=32.558 y=34.568 orient=2.8589], [x=31.528 y=32.011 orient=2.9834], [x=31.644 y=33.311 orient=2.9012], [x=31.644 y=33.311 orient=2.9012], [x=30.161 y=30.857 orient=2.9236], [x=30.161 y=30.857 orient=2.9236], [x=30.161 y=30.857 orient=2.9236], [x=31.627 y=32.659 orient=2.9054], [x=32.402 y=34.311 orient=2.8294], [x=30.434 y=30.197 orient=3.0222], [x=31.402 y=33.907 orient=2.8082], [x=31.402 y=33.907 orient=2.8082], [x=31.402 y=33.907 orient=2.8082], [x=28.859 y=29.602 orient=2.9684], [x=28.928 y=27.989 orient=3.1638], [x=29.007 y=28.230 o

* The orientations play a role, meaning get updated. Because you need to move the particles/robot before each resampling.